In [ ]:
import pandas as pd
from tbdynamics.constants import (
    age_strata,
    organ_strata,
    compartments,

)
from tbdynamics.plotting import plot_model_vs_actual
import arviz as az
import nevergrad as ng

# Import our convenience wrapper
from estival.wrappers.nevergrad import optimize_model
from tbdynamics.calib_utils import get_bcm, load_targets

In [ ]:
pd.options.plotting.backend = "plotly"

## Define Model

### Params and calibration targets

In [ ]:
params = {
    'treatment_duration': 0.5, # 6 months
    'screening_start_asymp': 0.,
}

bcm = get_bcm(params)

### Running Optimization

In [ ]:
opt_class = ng.optimizers.TwoPointsDE
orunner = optimize_model(bcm, opt_class=opt_class)
for i in range(8):
    rec = orunner.minimize(1000)
mle_params = rec.value[1]
mle_params
   

In [ ]:
mle_params = {'start_population_size': 2008109.4335554638,
 'contact_rate': 0.009892852470590574,
 'rr_infection_latent': 0.27813952895931426,
 'rr_infection_recovered': 0.11235460227287761,
 'progression_multiplier': 1.0235499973885789,
 'seed_time': 1890.4363736439068,
 'seed_num': 89.12769919853486,
 'seed_duration': 2.283779950521762,
 'smear_positive_death_rate': 0.3397878990882099,
 'smear_negative_death_rate': 0.03188729803490377,
 'smear_positive_self_recovery': 0.26700702728112635,
 'smear_negative_self_recovery': 0.20273850952310415,
 'screening_scaleup_shape': 0.09997351238071114,
 'screening_inflection_time': 1993.0429675569521,
 'screening_end_asymp': 0.6483939372547786}

In [ ]:
res = bcm.run(mle_params)
derived_df_0 = res.derived_outputs

In [ ]:
targets = load_targets()

### Outputs

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['pop'], "total_population", "Population", "Modelled vs Data"
)

In [ ]:
derived_df_0[[f"total_populationXage_{i}" for i in age_strata]].plot(
    title="Modelled populatation by age group", kind="area"
)

In [ ]:
incidence = pd.Series({2022: 176})
plot_model_vs_actual(
    derived_df_0, incidence, "incidence", "Incidence", "Modelled vs Data"
)

In [ ]:
derived_df_0["prevalence_infectious"].plot()

In [ ]:
derived_df_0[[f"prop_{compartment}" for compartment in compartments]].plot(kind="area")

In [ ]:
derived_df_0[[f"total_populationXorgan_{i}" for i in organ_strata]].plot(
    title="Modelled populatation by organ status", kind="area"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['notifs'], "notification", "Notification", "Modelled vs Data"
)

In [ ]:
plot_model_vs_actual(
    derived_df_0, targets['percentage_latent'], "percentage_latent", "Percentage latent", "Modelled vs Data"
)

In [ ]:
derived_df_0['cdr'].plot()